# Part 1 from tensorflow to HLS4ML


In [ ]:
import numpy as np
seed = 0
np.random.seed(seed)

import tensorflow as tf
tf.random.set_seed(seed)


import sys

sys.path.append("/home/webphy/Desktop/dnn_processor/")  # just to enable `dataset`
sys.path.append(
    "/home/webphy/Desktop/dnn_processor/dataset/"
)  # just to enable `dataset`

import dataset
import plotting

## Fetch the dataset


In [ ]:

BATCH_SIZE = 64
train_images_rgx = "../dataset/resized_dataset_texturas_v2/*/*.jpeg"
valid_images_rgx = "../dataset/resized_dataset_texturas_v2_test/*/*.jpeg"

train_ds, val_ds, nclasses = dataset.create_datasets(
    train_images_rgx, valid_images_rgx, BATCH_SIZE
)

## Load the model

In [ ]:
import tensorflow as tf
import keras

model_trial_id = 271

model = keras.models.load_model(
    f"../neural_network/automl/saved_models/striped_models/exp6_aug_calib/{model_trial_id}.h5"
)

# x = x_in = keras.layers.Input(shape=(256,256,3))
# for l in model.layers[1:]:
#     x = l(x)
#
#     if l.name == "activation_956":
#         break
# new_model = keras.models.Model(x_in, x)
# new_model.summary()

model.layers[-1].activation = keras.activations.linear
model.compile(
    "adam",
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

# model.summary()
new_model = model

## Check performance

In [ ]:
score = model.evaluate(val_ds)
print("Accuracy: {}".format(score[1]))

## Convert the model to FPGA firmware with hls4ml

### Make an hls4ml config & model

In [ ]:
import hls4ml

config = hls4ml.utils.config_from_keras_model(
    new_model,
    granularity="name",
    default_precision="fixed<4,4>",
    default_reuse_factor=2048,
    
)

config["Model"]["Strategy"] = "Resource"

for layer in config["LayerName"].keys():
    config["LayerName"][layer]["Strategy"] = "Resource"

config["LayerName"]["depthwise_conv2d_220"]["Strategy"] = "Latency"
config["LayerName"]["depthwise_conv2d_221"]["Strategy"] = "Latency"

config["LayerName"][f"dense_{model_trial_id}"]["ReuseFactor"] = 4000
config["LayerName"][f"dense_{model_trial_id}"]["Strategy"] = "Resource"


In [ ]:
print("-----------------------------------")
print("Configuration")
plotting.print_dict(config)
print("-----------------------------------")
hls_model = hls4ml.converters.convert_from_keras_model(
    new_model,
    hls_config=config,
    output_dir=f"model_{model_trial_id}/hls4ml_prj",
    part="xc7z020-clg400-1",
    io_type="io_stream",
)

Let's visualise what we created. The model architecture is shown, annotated with the shape and data types


In [ ]:
hls4ml.utils.plot_model(hls_model, show_shapes=True, show_precision=True, to_file=None)

## Compile, predict

Now we need to check that this model performance is still good. We compile the hls_model, and then use `hls_model.predict` to execute the FPGA firmware with bit-accurate emulation on the CPU.


In [ ]:
dataset__ = list(val_ds.as_numpy_iterator())
val_x = dataset__[0][0]
val_y = dataset__[0][1]

hls_model.compile()
val_ds_np = np.ascontiguousarray(val_x).astype(np.float32)
y_hls = hls_model.predict(val_ds_np)
y_hls = tf.nn.softmax(y_hls).numpy()

## Compare


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from matplotlib.lines import Line2D
from matplotlib.legend import Legend


classes = ["0", "1", "2", "3", "4"]

y_keras = new_model.predict(val_x)
y_keras = tf.nn.softmax(y_keras, axis=-1).numpy()

print(
    "Keras  Accuracy: {}".format(
        accuracy_score(np.argmax(val_y, axis=-1), np.argmax(y_keras, axis=-1))
    )
)
print(
    "hls4ml Accuracy: {}".format(
        accuracy_score(np.argmax(val_y, axis=1), np.argmax(y_hls, axis=1))
    )
)

fig, ax = plt.subplots(figsize=(9, 9))
_ = plotting.makeRoc(val_y, y_keras, classes)
plt.gca().set_prop_cycle(None)  # reset the colors
_ = plotting.makeRoc(val_y, y_hls, classes, linestyle="--")

lines = [Line2D([0], [0], ls="-"), Line2D([0], [0], ls="--")]

leg = Legend(ax, lines, labels=["keras", "hls4ml"], loc="lower right", frameon=False)
ax.add_artist(leg)

## Synthesize

**This can take several minutes.**

In [ ]:
hls_model.build(csim=False)

## Check the reports

Print out the reports generated by Vivado HLS. Pay attention to the Latency and the 'Utilization Estimates' sections


In [ ]:
hls4ml.report.read_vivado_report(f"model_{model_trial_id}/hls4ml_prj/")